# Module 8: 7 - Anatomy of an AI Agent - Tool Execution via OpenAI Function Calling
---------------------------------------------------------------------------------
In this lesson, we will enhance our AI Agent by enabling tool execution using [OpenAI's Function Calling](https://platform.openai.com/docs/guides/function-calling) capability. This feature allows the model to understand function signatures, detect when a tool needs to be used based on user intent, choose the right tool, and return the function signature to be executed by the AI Agent.

## Objectives
* Understand the concept of tool execution using OpenAI Function Calling.
* Update the LLM Client to accept and utilize tools.
* Integrate tool execution capabilities into the existing AI Agent structure.
* Test the AI Agent's ability to choose and execute tools based on user input.

## What this session covers:
* Installing necessary libraries for OpenAI integration.
* Defining the current agent structure, including the LLM Client and short-term memory.
* Implementing OpenAI's Function Calling for tool execution.
* Updating the agent to hold and use tools.
* Initializing and testing the enhanced AI Agent with example tools and interactions.

## Install Libraries

In [2]:
#! pip install openai

## Define Current Agent Structure

### LLM Client

In [1]:
from typing import Dict, Any, List
import openai

class OpenAIChatCompletion:
    """
    Interacts with OpenAI's API for chat completions.
    """
    def __init__(self, model: str, api_key: str = None, base_url: str = None):
        """
        Initialize with model, API key, and base URL.
        """
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def generate(self, messages: List[Dict], **kwargs) -> Dict[str, Any]:
        """
        Generate a response from input messages.
        """
        params = {'messages': messages, 'model': self.model, **kwargs}
        response = self.client.chat.completions.create(**params)
        return response.choices[0].message

### Short-Term Memory

In [2]:
from typing import List, Dict

class ChatMessageMemory:
    """Manages conversation context."""
    
    def __init__(self):
        self.messages = []
    
    def add_message(self, message: Dict):
        """Add a message to memory."""
        self.messages.append(message)
    
    def add_messages(self, messages: List[Dict]):
        """Add multiple messages to memory."""
        for message in messages:
            self.add_message(message)
    
    def get_messages(self) -> List[Dict]:
        """Retrieve all messages."""
        return self.messages.copy()
    
    def reset_memory(self):
        """Clear all messages."""
        self.messages = []

### Agent Base

In [3]:
class Agent:
    """Integrates LLM client, tools, and memory."""
    
    def __init__(self, llm_client, system_message: Dict[str, str], tools=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = ChatMessageMemory()
        self.system_message = system_message

    def run(self, user_message: Dict[str, str]):
        """Generate a response using LLM client and store context."""
        self.memory.add_message(user_message)
        chat_history = [self.system_message] + self.memory.get_messages()
        response = self.llm_client.generate(chat_history)
        self.memory.add_message(response)
        return response

## Implementing Function Calling

One reliable method for choosing tools and returning structured outputs is OpenAI's Function Calling. Introduced on June 13, 2023, this feature allows developers to describe functions to models trained to generate a JSON object with the necessary arguments based on user input.

### Update OpenAI Client to Accept Tools

In [19]:
from typing import Dict, Any, List
import openai

class OpenAIChatCompletion:
    """Interacts with OpenAI's API for chat completions."""
    def __init__(self, model: str, api_key: str = None, base_url: str = None):
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def generate(self, messages: List[str], tools: List[Dict[str, Any]] = None, **kwargs) -> Dict[str, Any]:
        """Generates a response from OpenAI's API."""
        params = {'messages': messages, 'model': self.model, 'tools': tools, **kwargs}
        response = self.client.chat.completions.create(**params)
        return response.choices[0].message

### Update Agent to Hold Tools

In [31]:
from typing import Dict

class Agent:
    """Integrates LLM client, tools, and memory."""
    def __init__(self, llm_client, system_message: Dict[str, str], tools=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = ChatMessageMemory()
        self.system_message = system_message

    def run(self, user_message: Dict[str, str]):
        self.memory.add_message(user_message)
        chat_history = [self.system_message] + self.memory.get_messages()
        response = self.llm_client.generate(chat_history, tools=self.tools)
        self.memory.add_message(response)
        return response

### Testing Current Agent

Define Dummy Functions

In [50]:
import random

def get_weather(location: str) -> str:
    """Gets weather information."""
    temperature = random.randint(60, 80)
    return f"{location}: {temperature}F."

def jump(distance: str) -> str:
    """Jumps a specific distance."""
    return f"I jumped the following distance {distance}"

Define Python Function Signature in OAI Function Calling Format

In [51]:
get_weather_func_dict = {
    'type': 'function',
    'function': {
        'name': 'get_weather',
        'description': 'Get weather information based on location.',
        'parameters': {
            'properties': {'location': {'type': 'string'}},
            'required': ['location'],
            'type': 'object'
        }
    }
}

jump_func_dict = {
    'type': 'function',
    'function': {
        'name': 'jump',
        'description': 'Jump a specific distance.',
        'parameters': {
            'properties': {'distance': {'type': 'string'}},
            'required': ['distance'],
            'type': 'object'
        }
    }
}

tools = [get_weather_func_dict, jump_func_dict]

Initialize Client

In [52]:
# API from environment variable
# import os
# api_key = os.getenv("OPENAI_API_KEY"))

api_key=""

client = OpenAIChatCompletion(
    base_url='https://api.openai.com/v1',
    model='gpt-4',
    api_key=api_key
)

Define System messages

In [53]:
# Define the system message
system_message = {"role": "system", "content": "You are a weather assistant."}

Initialize Agent

In [54]:
# Initialize the Agent with the LLM client, system message, and tools
agent = Agent(llm_client=client, system_message=system_message, tools=tools)

Send a User Message

In [55]:
# Define a user message
user_message = {"role": "user", "content": "What is the weather in Virginia?"}

# Generate a response using the agent
response = agent.run(user_message)
response

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_a9uSJpANwylDReIngvjwdcYg', function=Function(arguments='{\n  "location": "Virginia"\n}', name='get_weather'), type='function')])

In [56]:
response.to_dict()

{'content': None,
 'role': 'assistant',
 'tool_calls': [{'id': 'call_a9uSJpANwylDReIngvjwdcYg',
   'function': {'arguments': '{\n  "location": "Virginia"\n}',
    'name': 'get_weather'},
   'type': 'function'}]}

## Parse Tool Response

In [57]:
import json

tool_response = response.tool_calls[0]
tool_arguments = json.loads(tool_response.function.arguments)
tool_arguments

{'location': 'Virginia'}

## Execute Tools

In [58]:
tool_execution_results = get_weather(**tool_arguments)
tool_execution_results

'Virginia: 64F.'